In [83]:
import imp
environment = imp.load_source('environment','../../smarty/environment.py')

### Initialize Each Environment type

* Atom
* Bond
* Angle
* Torsion
* Improper 


In [84]:
atomEnv = environment.AtomChemicalEnvironment()
bondEnv = environment.BondChemicalEnvironment()
angleEnv = environment.AngleChemicalEnvironment()
torsionEnv = environment.TorsionChemicalEnvironment()
impropEnv = environment.ImproperChemicalEnvironment()

EnvList = [atomEnv, bondEnv, angleEnv, torsionEnv, impropEnv]
names = ['atom','bond','angle','torsion','improper']

for idx, Env in enumerate(EnvList):
    print("%10s: %s" % (names[idx], Env.asSMIRKS()))

      atom: [*:1]
      bond: [*:2]~[*:1]
     angle: [*:2](~[*:3])~[*:1]
   torsion: [*:3](~[*:2]~[*:1])~[*:4]
  improper: [*:4]~[*:2](~[*:1])~[*:3]


### Add descriptions to atoms

You can add descriptive features to atoms, you can chose between

* `bases` (thing that are OR'd together) or 
* `decorators` (things that are AND'd together) 

Here we'll use torsionEnv as an example since it has more random bonds and atoms to chose between

In [85]:
# create new torsion:
torsionEnv = environment.TorsionChemicalEnvironment()

# Select and atom
pickAtom = torsionEnv.selectAtom()
print "initially: ", torsionEnv.asSMIRKS()
print "chose atom", pickAtom.asSMIRKS()

pickAtom.addBase('#1')
print torsionEnv.asSMIRKS(), "torsion after adding '#1' to atom base" 

pickAtom.addDecorator('H0')
print torsionEnv.asSMIRKS(), "torsion after adding 'H0' to atom decorators"

initially:  [*:3](~[*:2]~[*:1])~[*:4]
chose atom [*:4]
[*:3](~[*:2]~[*:1])~[#1:4] torsion after adding '#1' to atom base
[*:3](~[*:2]~[*:1])~[#1;H0:4] torsion after adding 'H0' to atom decorators


### Add same descriptions to bonds

Bonds have the same properties
bonds however are defined by the atoms they connect so when selecting a bond you get three items

* atom1 (atom on one end of bond)
* atom2 (atom on the otehr end of the bond)
* bond (bond object)


In [86]:
# create new torsion
torsionEnv = environment.TorsionChemicalEnvironment()

# pick a bond
atom1, atom2, pickBond = torsionEnv.selectBond()

print "initially: ", torsionEnv.asSMIRKS()
print "chose bond connecting %s %s %s" % (atom1.asSMIRKS(), pickBond.asSMARTS(), atom2.asSMIRKS())

pickBond.addBase('-')
print torsionEnv.asSMIRKS(), "torsion after adding '-' to bond base" 

pickBond.addDecorator('@')
print torsionEnv.asSMIRKS(), "torsion after adding '@' to bond decorators"
# Note now the bond between 1 and 2 is - instead of ~

initially:  [*:4]~[*:3]~[*:2]~[*:1]
chose bond connecting [*:1] ~ [*:2]
[*:4]~[*:3]~[*:2]-[*:1] torsion after adding '-' to bond base
[*:4]~[*:3]~[*:2]-;@[*:1] torsion after adding '@' to bond decorators


### Add a new atom 

Specify an atom to attach a new atom. 
You can provide descriptors 
(bases that are or'd together or decorators that are and'd to the end) 
for the atom and/or bond or leave them empty. 

The bases and decorators should be lists of strings, 
either way they will be used to create the new Atom and new Bond objects.

The `addAtom` method returns the created atom.

In [87]:
# create new torsion
torsionEnv = environment.TorsionChemicalEnvironment()

bondToAtom = torsionEnv.selectAtom()
print("Bond to atom %s" % bondToAtom.asSMIRKS())

atomBases = ['#6']
atomDecorators = ['a']
bondBases = [':']
bondDecorators = None

# addAtom(self, bondToAtom, bondBases=None, bondDecorators=None, newBases=None, newDecorators=None, newAtomIndex=None)
newAtom = torsionEnv.addAtom(bondToAtom, bondBases, bondDecorators, atomBases, atomDecorators, None)
print("New atom (%s) in torsion %s" % (newAtom.asSMIRKS(), torsionEnv.asSMIRKS()))

Bond to atom [*:1]
New atom ([#6;a]) in torsion [*:2](~[*:3]~[*:4])~[*:1]:[#6;a]


### Remove an atom

Removing an atom removes the specified atom if it meets the conditions. There are three possible outcomes. 

* The specified atom is a labeled atom so it cannot be removed
* The specified atom connectes two other atoms so it cannot be removed
* The specified atom is removed

Below are examples for each of these scenarios with a Bond type environment

In [88]:
bondEnv = environment.BondChemicalEnvironment()
# atom1 is the first first atom in the bond
bondEnv.removeAtom(bondEnv.atom1)

Cannont remove labeled atom [*:1]


In [89]:
bondEnv = environment.BondChemicalEnvironment()
newAtom1 = bondEnv.addAtom(bondEnv.atom1, None, None, ['#6'], None)
newAtom2 = bondEnv.addAtom(newAtom1, None, None, ['#8'], None)
print bondEnv.asSMIRKS()

# Try removing atom 1
bondEnv.removeAtom(newAtom1)

# Remove atom 2, show new SMIRKS
bondEnv.removeAtom(newAtom2)
print bondEnv.asSMIRKS()

[#8]~[#6]~[*:1]~[*:2]
Cannot remove atom [#6] because it connects two atoms
[#6]~[*:1]~[*:2]


### AsSMARTS

This will be especially helpful for comparing this tool with our string manipulations for smarty with atom types
It returns the SMARTS string for atom1 as a single atom. 
This will allow us to use the same comparisons we are using now in testing the chemical environment tool


In [95]:
# create atom
atomEnv = environment.AtomChemicalEnvironment(['#1'])
print atomEnv.asSMARTS()
carbon = atomEnv.addAtom(atomEnv.atom1, '-', None, ['#6'])
print atomEnv.asSMARTS()
oxygen1 = atomEnv.addAtom(carbon, '-', None, ['#8'])
oxygen2 = atomEnv.addAtom(carbon, '-', None, ['#8'])
oxygen3 = atomEnv.addAtom(carbon, '-', None, ['#8'])
print atomEnv.asSMARTS()

[#1]
[#1$(*-[#6])]
[#1$(*-[#6](-[#8])(-[#8])-[#8])]
